# Slots

Veamos el tema de hacer slots para todo el dataset 

In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import django
from tqdm.auto import tqdm

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

sys.path.append("..")
os.environ['DJANGO_SETTINGS_MODULE'] = 'news_labelling.settings'
django.setup()

from api.models import Comment, ArticleLabel, CommentLabel, Article, Assignment
from django.contrib.auth.models import User

In [2]:
import json

with open("../data/training_ids.json") as f:
    training_ids = json.load(f)
    
training_articles = Article.objects.filter(tweet_id__in=training_ids)

training_articles.count()

14

¿Cuáles quedaron fuera?


In [3]:
out_ids = [art_id for art_id in training_ids if not Article.objects.filter(tweet_id=art_id).count()]

out_ids

[1296651850796732419,
 1263476271603998720,
 1255472854776393728,
 1275218345969831936,
 1300878739870539776,
 1293587848277434370,
 1238231799995736066,
 1308802398752854016,
 1281715363798036490,
 1265837173510672390,
 1249151575257481227]

Los que quedaron fuera: (falta uno creo!)

- Verónica Magario pidió endurecer la cuarentena: “Debemos frenar la circulación de personas en el AMBA para evitar contagios”
- Según un experto italiano, “el coronavirus se está debilitando y podría morir solo sin una vacuna”
- Al borde de las lágrimas, Luis Brandoni convocó un banderazo en contra del Gobierno, pero “cumpliendo todos los protocolos”
- Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria"
- Video: Jair Bolsonaro le hace upa a un hombre de talla baja pensando que era un niño
- El proyecto del impuesto a la riqueza “está listo” y se presentará la próxima semana
- Ante el cierre de los cines por la pandemia el presidente del INCAA quiere ponerle otro impuesto a Netflix
- Filminas para todos, menos para Maduro
- Sergio Berni: “Debemos ir a una cuarentena absoluta, mucho más rigurosa que al principio y sin transporte público”
- Intentó violar a su novia y ella lo asesinó a escobazos

Espectacular! Se fueron todas las notas malas!!! :-)

In [4]:
for art in training_articles:
    print(f"{art.tweet_id} -- {art.title}")

1242447731286777856 -- El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón
1243147648238059520 -- Escándalo en España: fallan los tests de coronavirus fabricados en China
1247580175455444993 -- Condenan a ocho años de cárcel por corrupción al ex presidente ecuatoriano Rafael Correa
1263868365468135430 -- Los piratas del siglo XXI se roban US$1000 millones por año del mar argentino
1273676406614978561 -- "Evolucionen": Romina Malaspina hizo un fuerte descargo tras los comentarios sexistas por su look en el noticiero
1273719287614787584 -- Verónica Magario pidió endurecer la cuarentena: “Debemos frenar la circulación de personas en el AMBA para evitar contagios”
1277424269153239041 -- Ginés González García admite que prohibieron a los runners por una cuestión de imagen “más que por los contagios”
1294980385907015680 -- ¿Terminará Argentina como Venezuela?
1296244483655454720 -- Video: Jair Bolsonaro le hace upa a un hombre de talla baja pensando que era un n

## Demo ids

In [7]:

with open("../data/demo_ids.json") as f:
    ids = json.load(f)



demo_articles = Article.objects.filter(tweet_id__in=ids)

print("Collected article_ids")

print([art.tweet_id for art in demo_articles])

for art in demo_articles:
    print("\n"*2, "-"*80)
    
    print(art, "\n"*2,"\n")
    print(art.tweet_id)
    for i, comm in enumerate(art.comment_set.all()):
        print(f"{i+1:<5} - ",comm.text)


    

Collected article_ids
[1242419653030313984, 1253699106385481728, 1254951309469519875, 1275389016511336448, 1310165736011239424]


 --------------------------------------------------------------------------------
@infobae: Mariana Genesio Peña pasa su cuarentena total con guantes, barbijo y desnuda: “Mi cuerpo es el planeta Tierra” 

 

1242419653030313984
1     -  @usuario Y al que no le guste, que te chupe un huevo... o los dos 😂
2     -  @usuario Jajajaa que pelotuda!!!!.
3     -  @usuario Mariana Genesio pasa la cuarentena rascándose los huevos...
4     -  @usuario Con 3 piernas cualquiera es feliz!!!
5     -  @usuario Bue apostemos al morno de ver una trans en bolas .... que seriedad Che
6     -  @usuario Que desinfecte el picaporte. 😂😂😂
7     -  @usuario Los diputados aprovechan para embucharse $ 100.000 y ésta chica para primocionarse
8     -  @usuario Dios misericordioso, no nos des más chances, tirá de una vez el rayo exterminador, estamos perdidos. Q manera de leer boludeces y

## Interview Articles


In [8]:
art = Article.objects.all()[0]

In [9]:
type(art).__name__

'Article'

In [11]:
import json
with open("../data/interview_ids.json") as f:
    ids = json.load(f)

interview_articles = Article.objects.filter(tweet_id__in=ids)

print("Collected article_ids")

print([art.tweet_id for art in interview_articles])

for art in interview_articles:
    print("\n"*2, "-"*80)
    
    print(art, "\n"*2,"\n")
    continue
    for i, comm in enumerate(art.comment_set.all()):
        print(f"{i+1:<5} - ",comm.text)


    

Collected article_ids
[1242447731286777856, 1243147648238059520, 1254951309469519875, 1296507657189314566, 1302741866681491456]


 --------------------------------------------------------------------------------
@infobae: El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón 

 



 --------------------------------------------------------------------------------
@infobae: Escándalo en España: fallan los tests de coronavirus fabricados en China 

 



 --------------------------------------------------------------------------------
@infobae: Thelma Fardin contó cómo sigue la causa contra Juan Darthés: “Brasil le permite la impunidad” 

 



 --------------------------------------------------------------------------------
@infobae: Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria” 

 



 --------------------------------------------------------------------------------
@LANACION: Luis Novaresio contó cómo será su boda con Braul

In [7]:
import random

all_articles = list(Article.objects.all())

sample_articles = random.sample(all_articles, 10)

In [9]:
for art in sample_articles:
    print("\n"*2, "-"*80)
    print(art.tweet_id, art, "\n"*2, art.metadata, "\n")
    for comm in art.comment_set.all()[:5]:
        print(comm.text)
    
    



 --------------------------------------------------------------------------------
1293660570240143360 @clarincom: “Les pibis”, la expresión con lenguaje inclusivo de un joven funcionario nacional que estalló en las redes 

 {'trolo -"no seas" -"no sean"': 2, '"planero"': 1, 'negro -"en negro" -"Rio" -"peste" -"humor negro"': 1} 

@usuario yo pensé que era  lis pibis.
@usuario Gobierno de mierda ,con ideas de mierda para robar  y a la mierda todo !
@usuario Que mamarracho que son
@usuario Cada que incluya con la vocal que se le antoje, menos la o.
@usuario Todo lo que tocan lo destruyen, también van por la lengua española que es tan rica y maravillosa!!!!


 --------------------------------------------------------------------------------
1235579313224900610 @clarincom: Mauricio Macri fue escrachado en un avión: “Ladrón, arruinaste el país” 

 {'feministas': 26, 'aborteras': 1, 'trolo -"no seas" -"no sean"': 2, '"sucios"': 1, '"una bomba"': 1, '"planero"': 2, 'negro -"en negro" -"Rio" 